# Function to query hoteles.com

The following function **extract** gets data from the page hoteles.com for the web scrapping project

These are the libraries nedded for the function:

In [1]:
import time
import requests
from bs4 import BeautifulSoup as BS
from wolframclient.evaluation import WolframLanguageSession
from wolframclient.language import wl, wlexpr, Global

In [2]:
def extract(wolfram_session, web_browser, place, start_date, end_date, order='STAR_RATING_HIGHEST_FIRST', rooms={1:{'adults':2}}):
    
    url = 'https://www.hoteles.com/search.do?$place$&q-check-in=$start_date$&q-check-out=$end_date$&$rooms$&sort-order=$order$'
        
    # Repalce the input of the function in the url
    url = url.replace('$place$', place)
    url = url.replace('$start_date$', start_date)
    url = url.replace('$end_date$', end_date)
    url = url.replace('$order$', order)
    
    # Replce rooms
    rooms_url = 'q-rooms='+str(len(rooms.keys()))
    i=0
    for a in rooms.values():
        rooms_url = rooms_url + '&' + 'q-room-' + str(i) + '-adults=' + str(a['adults'])
        i= i + 1
    rooms_url = rooms_url + '&' + '&'.join(['q-room-'+str(n)+'-children=0' for n in range(len(rooms.keys()))])
    url = url.replace('$rooms$', rooms_url)
    
    print(url)
    
    # Open the web page in a browser to charge dynamical content. 
    # This will be donde using Wolfram (instead of more traditional packages like Selenium)
    wolfram_session.evaluate(wl.WebExecute(web_browser, wl.Rule("OpenPage", url))) # go to the url
    # scroll down using javascript code because some dynamical content is charged after scrolling down
    wolfram_session.evaluate(wl.WebExecute(web_browser, wl.Rule('JavascriptExecute', 'window.scrollTo(0,2000);')))
    time.sleep(3.1416)
    # get the html content using Javascript code
    html = wolfram_session.evaluate(wl.WebExecute(web_browser, wl.Rule('JavascriptExecute',\
                                                                          'return document.documentElement.outerHTML;')))  
    # Return as Beautifull Soup
    soup = BS(html)
    return soup

The function assumes that a wolfram session and a browser have been started like the following:

In [3]:
session = WolframLanguageSession() # start a Wolfram Session
session.evaluate(wlexpr('browser = StartWebSession["Firefox"]')) # start a browser

WebSessionObject[{'SessionID': 'd78ddd12-4e35-4e3d-b2b9-9637fe02603a', 'Browser': 'Firefox', 'URL': 'http://localhost:52598', 'Exec': 'C:\\Program Files\\Wolfram Research\\Mathematica\\12.3\\SystemFiles\\Components\\WebUnit\\Resources\\DriverBinaries\\GeckoDriver\\Windows-x86-64\\geckodriver.exe', 'Process': ProcessObject[{'ManagedProcess': True, 'UID': 0, 'PID': 13648, 'PPID': 7120, 'Program': 'geckodriver.exe', 'Path': 'C:\\Program Files\\Wolfram Research\\Mathematica\\12.3\\SystemFiles\\Components\\WebUnit\\Resources\\DriverBinaries\\GeckoDriver\\Windows-x86-64\\geckodriver.exe', 'User': 'truma', 'StartTime': DateObject[(2021, 10, 27, 17, 36, 49.0), 'Instant', 'Gregorian', -5.0]}], 'Active': True, 'SessionTime': Decimal('3844345013.1783006')}]

After the use of the funciton the sessions can be closed as:

In [12]:
session.evaluate(wl.DeleteObject(Global.browser))
session.terminate()

To see further information about the Wolfram package see:

https://reference.wolfram.com/language/WolframClientForPython/#:~:text=The%20Wolfram%20Client%20Library%20for,or%20a%20private%20Wolfram%20Cloud).

To get a free Wolfram engine (nedded to use the wolfram package) see:

https://www.wolfram.com/engine/

The following dictionary can help to order the results, they should be used in the order parameter of extract:

In [4]:
orders = {'star desc':'STAR_RATING_HIGHEST_FIRST', 'stars asc':'STAR_RATING_LOWEST_FIRST', 'featured':'BEST SELLER',
          'guest rating':'GUEST_RATING', 'price desc':'PRICE_HIGHEST_FIRST', 'price asc':'PRICE'}

## Examples

### Example 1

The following example passes as the first 2 arguments the Wolfram and browser sessions previously loaded. Then it puts the destination id of Baños, then check-in and check-out dates. Then, specifies the rooms (including the number of rooms and the number of adults in each room). Finally, the order of the result is specified using the orders dictionary:

In [6]:
soup = extract(session, Global.browser, 'destination-id=429944', '2021-10-30', '2021-10-31', rooms={1:{'adults':1}, 2:{'adults':1}}, order=orders['price desc'])

https://www.hoteles.com/search.do?destination-id=429944&q-check-in=2021-10-30&q-check-out=2021-10-31&q-rooms=2&q-room-0-adults=1&q-room-1-adults=1&q-room-0-children=0&q-room-1-children=0&sort-order=PRICE_HIGHEST_FIRST


In [7]:
print(soup.prettify())

<html class="js intl-es_EC" dir="LTR" lang="es">
 <head>
  <title>
   Hoteles.com - hoteles en Baños de Agua Santa, Tungurahua, Ecuador
  </title>
  <meta charset="utf-8"/>
  <meta content="text/html;charset=utf-8" http-equiv="content-type"/>
  <meta http-equiv="X-UA-Compatible" name="IE=Edge"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <meta content="notranslate" name="google"/>
  <meta content="Hotels.com - Deals &amp; Discounts for Hotel Reservations from Luxury Hotels to Budget Accommodations" name="title"/>
  <meta content="reservaciones hoteles online, ofertas hoteles, hoteles baratos, hoteles" name="keywords"/>
  <meta content="Hoteles.com | Encuentra hoteles baratos y descuentos al reservar en Hoteles.com. Compara ofertas y promociones, y lee comentarios imparciales sobre los hoteles." name="description"/>
  <meta content="#ffffff" name="theme-color"/>
  <link href="https://a.cdn-hotels.com/kesstatic/b593c2583e9df19f1490.png" rel="shortcut icon"/

The text containing the list of hotels can be extracted as follows:

In [9]:
info_place = soup.find_all('ul', {'class':'_3f26d2'})   
raw_data = info_place[0].getText('\n')
print(raw_data)

Samari Spa Resort
VIP
Establecimientos VIP
Los establecimientos VIP se distinguen por ofrecer un servicio excepcional a nuestros clientes.
Avenida De Las Amazonas, Baños de Agua Santa, Tungurahua
Baños de Agua Santa
Centro de la ciudad a 11 km
Cascada Pailón del Diablo a 14 km
9,0
.
Magnífico
34 comentarios de huéspedes de Hoteles.com

Piscina

Estacionamiento gratis

Traslado desde/hacia el aeropuerto

Spa

Gimnasio

Restaurante
USD 213
por habitación
USD 426 por 2 habitaciones
Acumula sellos
Acumula 10 sellos y recibe una noche de regalo* con Hoteles.com™ Rewards
Reservar
Samari Spa Resort
Se abre en una nueva ventana
Ayamtai Jea Guest House
Juan Leon Mera y los Misioneros, Baños de Agua Santa
Baños de Agua Santa
Centro de la ciudad a 13 km
Cascada Pailón del Diablo a 16 km

Estacionamiento gratis
Quedan 3 a
USD 120
por unidad
USD 240 por 2 cabañas
Acumula sellos
Acumula 10 sellos y recibe una noche de regalo* con Hoteles.com™ Rewards
Reservar
Ayamtai Jea Guest House
Se abre e

### Example 2

The function takes 5 mandatory input arguments (wolfram_session, web_browser, place, check in data, check out date) and 2 optional arguments (rooms, order). Folllowing a description of the input parameters:

1. The wolfram session 

2. The browser session

3. The argument place is the codification of a place that hoteles.com makes.

4. Check in date must follow the standard yyyy-mm-dd

5. Check out parameter, must follow the standard yyyy-mm-dd.

6. The rooms an how many adults are in each room as a dict

7. order (orders can help to specify this)

The following example gets info from Baños (Baños is specified by hoteles.com as 'destination-id=429944'):

In [10]:
html = extract(session, Global.browser, 'destination-id=429944', '2021-10-30', '2021-11-03')
print(html.prettify())

https://www.hoteles.com/search.do?destination-id=429944&q-check-in=2021-10-30&q-check-out=2021-11-03&q-rooms=1&q-room-0-adults=2&q-room-0-children=0&sort-order=STAR_RATING_HIGHEST_FIRST
<html class="js intl-es_EC" dir="LTR" lang="es">
 <head>
  <title>
   Hoteles.com - hoteles en Baños de Agua Santa, Tungurahua, Ecuador
  </title>
  <meta charset="utf-8"/>
  <meta content="text/html;charset=utf-8" http-equiv="content-type"/>
  <meta http-equiv="X-UA-Compatible" name="IE=Edge"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <meta content="notranslate" name="google"/>
  <meta content="Hotels.com - Deals &amp; Discounts for Hotel Reservations from Luxury Hotels to Budget Accommodations" name="title"/>
  <meta content="reservaciones hoteles online, ofertas hoteles, hoteles baratos, hoteles" name="keywords"/>
  <meta content="Hoteles.com | Encuentra hoteles baratos y descuentos al reservar en Hoteles.com. Compara ofertas y promociones, y lee comentarios imparcial

### Example 3

The following query gets information for hotels in Madrid:

In [11]:
l = extract(session, Global.browser, 'destination-id=457987', '2021-10-31', '2021-11-03')
info_place = l.find_all('ul', {'class':'_3f26d2'})   
raw_data = info_place[0].getText('\n')
print(raw_data)

https://www.hoteles.com/search.do?destination-id=457987&q-check-in=2021-10-31&q-check-out=2021-11-03&q-rooms=1&q-room-0-adults=2&q-room-0-children=0&sort-order=STAR_RATING_HIGHEST_FIRST
Precio secreto disponible
Acceder
Relais & Châteaux Hotel Orfila
5 estrellas
VIP
Establecimientos VIP
Los establecimientos VIP se distinguen por ofrecer un servicio excepcional a nuestros clientes.
Calle de Orfila, 6, Madrid, 28010, Madrid
Almagro
Centro de la ciudad a 1,4 km
IFEMA (Feria de Madrid) a 8,2 km
9,8
.
Excepcional
89 comentarios de huéspedes de Hoteles.com

Estacionamiento gratis

Se aceptan mascotas

Traslado desde/hacia el aeropuerto

Restaurante

Bar

Bañera
15% de desc.
Quedan 2 a
USD 723
El precio tachado se basa en la tarifa estándar en nuestro sitio, determinada por el establecimiento. 
USD 614
por noche
USD 1,843 por 3 noches
Acumula sellos
Acumula 10 sellos y recibe una noche de regalo* con Hoteles.com™ Rewards
Reservar

Cancelación gratis

Pago en el establecimiento disponi